In [1]:
import os
import re
from pathlib import Path
from PyPDF2 import PdfReader
from market_agents.memory.config import load_config_from_yaml
from market_agents.memory.setup_db import DatabaseConnection
from market_agents.memory.embedding import MemoryEmbedder
from market_agents.memory.knowledge_base import MarketKnowledgeBase
from market_agents.memory.knowledge_base_agent import KnowledgeBaseAgent
from market_agents.memory.vector_search import MemoryRetriever
from market_agents.agents.market_agent import MarketAgent
from market_agents.inference.message_models import LLMConfig
from market_agents.environments.mechanisms.chat import ChatEnvironment


In [2]:
# Load config
config_path = Path("../market_agents/memory/memory_config.yaml")
memory_config = load_config_from_yaml(str(config_path))

# Initialize database connection and embedding service
db_conn = DatabaseConnection(memory_config)
embedder = MemoryEmbedder(memory_config)

# Initialize knowledge base with "shakespeare" prefix
shakespeare_kb = MarketKnowledgeBase(
    config=memory_config,
    db_conn=db_conn,
    embedding_service=embedder,
    table_prefix="shakespeare_kb"
)

In [3]:
def document_reader(file_path: str) -> str:
    """
    Read content from PDF or TXT files and return cleaned text
    
    Args:
        file_path (str): Path to the document file
        
    Returns:
        str: Cleaned text content from the document
    """
    text_content = ""
    
    if file_path.endswith('.pdf'):
        pdf_reader = PdfReader(file_path)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            cleaned_text = re.sub(r'FTLN \d+', '', page_text)
            cleaned_text = re.sub(r'^\d+\s*', '', cleaned_text, flags=re.MULTILINE)
            text_content += cleaned_text
    
    elif file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text_content = f.read()
    else:
        raise ValueError("Unsupported file format. Please provide a PDF or TXT file.")
            
    return text_content

# Read document content
#file_path = "/Users/interstellarninja/Documents/projects/MarketAgents/hamlet_PDF_FolgerShakespeare.pdf"
file_path = "/Users/interstellarninja/Documents/projects/MarketAgents/examples/the_complete_works_of_william_shakespear.txt"
text_content = document_reader(file_path)

In [ ]:
# Store in knowledge base with metadata
knowledge_id = shakespeare_kb.ingest_knowledge(
    text_content,
    metadata={
        "source": "The Complete Works of William Shakespeare",
        "type": "play",
        "author": "William Shakespeare"
    }
)
print(f"Successfully created Hamlet knowledge base with ID: {knowledge_id}")

In [3]:
# Initialize retriever
retriever = MemoryRetriever(memory_config, db_conn, embedder)

# Create knowledge base agent
kb_agent = KnowledgeBaseAgent(
    market_kb=shakespeare_kb,
    retriever=retriever
)

In [ ]:
# Test some queries
def query_hamlet(question: str):
    print(f"\nQuestion: {question}")
    print("-" * 80)
    
    results = kb_agent.retrieve(question, "shakespeare_kb")
    
    for i, chunk in enumerate(results, 1):
        print(f"\nResult {i} (Similarity: {chunk.similarity:.3f})")
        print("Text:", chunk.text)
        print("\nContext:", chunk.context)
        print("-" * 80)

# Try different queries
queries = [
    "What is Hamlet's famous 'To be or not to be' soliloquy about?",
    "How does Ophelia die in the play?",
    "What is the role of the ghost in the play?",
    "Describe the relationship between Hamlet and Horatio"
]

for query in queries:
    query_hamlet(query)

In [4]:
import json


chat_env = ChatEnvironment(name="shakespeare_chat")

shakespeare_agent = MarketAgent.create(
    memory_config=memory_config,
    db_conn=db_conn,
    agent_id="shakespeare_agent",
    use_llm=True,
    llm_config=LLMConfig(
        client="openai",
        model="gpt-4o",
        temperature=0.5,
        max_tokens=1024
    ),
    knowledge_agent=kb_agent, 
    environments={"chat": chat_env},
)

# Add a test message to the chat
shakespeare_agent.task = "What is Hamlet's famous 'To be or not to be' soliloquy about?"
chat_env.mechanism.add_user_message(shakespeare_agent.task)

# Generate a response
perception = await shakespeare_agent.perceive(environment_name="chat")
print(json.dumps(perception, indent=4))

action = await shakespeare_agent.generate_action(environment_name="chat", perception=perception)
print(json.dumps(action, indent=4))

reflection = await shakespeare_agent.reflect(environment_name="chat")
print(json.dumps(reflection, indent=4))



Cognitive Memory Results:
Memory 1:
{'cognitive_step': 'action', 'content': '{"agent_id": "agent_007", "action": {"thoughts": [{"reasoning": "The user has asked about Hamlet\'s \'To be or not to be\' soliloquy, which is a significant part of the play \'Hamlet\' by William Shakespeare. This soliloquy explores existential themes, pondering the nature of existence and the question of life and death."}, {"reasoning": "The documents retrieved are from \'Antony and Cleopatra\', which is a different play by Shakespeare and does not directly relate to the user\'s question about Hamlet."}, {"reasoning": "To address the user\'s query effectively, I will provide a concise summary of the \'To be or not to be\' soliloquy, highlighting its main themes and significance."}, {"reasoning": "I will also clarify the discrepancy regarding the retrieved documents to ensure the user understands why the content they received was not relevant to their question."}], "final_answer": "Hamlet\'s \'To be or not to

In [6]:
shakespeare_agent.interactions

[{'id': 'shakespeare_agent',
  'name': 'agent',
  'system': {'role': 'system',
   'content': 'Role: You are a agent agent. You are a long-horizon task execution agent based on verbal reinforcement learning. The current date and time is 2025-01-23 05:55:46.\n\n'},
  'task': 'Tasks: Your are assigned with following tasks:\nPerceive the current state of the chat environment:\n\n## Environment State\n<environment_state>\n-   content: What is Hamlet\'s famous \'To be or not to be\' soliloquy about?\n  timestamp: 2025-01-23 05:55:45\n  role: user\n</environment_state>\n\n## Recent Cognitive Memories\n<cognitive_memories>\n-   cognitive_step: action\n  content: {"agent_id": "agent_007", "action": {"thoughts": [{"reasoning": "The user has asked about Hamlet\'s \'To be or not to be\' soliloquy, which is a significant part of the play \'Hamlet\' by William Shakespeare. This soliloquy explores existential themes, pondering the nature of existence and the question of life and death."}, {"reasoning

In [7]:
shakespeare_agent.environments

{'chat': ChatEnvironment(name='shakespeare_chat', address=None, current_step=0, max_steps=10, action_space=ChatActionSpace(allowed_actions=[<class 'market_agents.environments.mechanisms.chat.ChatAction'>]), observation_space=ChatObservationSpace(allowed_observations=[<class 'market_agents.environments.mechanisms.chat.ChatObservation'>]), history=EnvironmentHistory(steps=[]), mechanism=ChatMechanism(sequential=True, chat_history=[ChatMessage(content="What is Hamlet's famous 'To be or not to be' soliloquy about?", timestamp='2025-01-23 05:55:45', role='user')]))}

In [8]:
shakespeare_agent.knowledge_agent

KnowledgeBaseAgent(market_kb=<market_agents.memory.knowledge_base.MarketKnowledgeBase object at 0x120872cf0>, retriever=<market_agents.memory.vector_search.MemoryRetriever object at 0x1217e76b0>, knowledge_bases={}, retrieved_knowledge=[])

In [9]:
shakespeare_agent.last_perception

{'monologue': "The user has inquired about Hamlet's 'To be or not to be' soliloquy, a profound reflection on existential themes. While the retrieved documents are excerpts from 'Antony and Cleopatra,' which are unrelated to Hamlet, I have consistently addressed this by providing a detailed summary of the soliloquy. This ensures the user receives relevant information despite the document mismatch. It's important to maintain transparency and user trust by acknowledging the discrepancy and focusing on the user's original query. 😊📚",
 'key_observations': ["User asked about Hamlet's 'To be or not to be' soliloquy.",
  "Retrieved documents are from 'Antony and Cleopatra,' not directly related to the user's question."],
 'strategy': ["Provide a summary of Hamlet's soliloquy focusing on its themes of existence, life, and death.",
  "Clarify any confusion by stating that the retrieved documents are not relevant to the user's question.",
  'Ensure transparency about document discrepancies to mai

In [10]:
shakespeare_agent.last_action

{'agent_id': 'shakespeare_agent',
 'action': {'thoughts': [{'reasoning': "The user has asked about Hamlet's 'To be or not to be' soliloquy. This is a significant part of the play 'Hamlet' by William Shakespeare, exploring existential themes, pondering the nature of existence and the question of life and death. It's important to provide a clear and concise summary of this soliloquy to address the user's query."},
   {'reasoning': "The documents retrieved are from 'Antony and Cleopatra', which is a different play by Shakespeare and does not directly relate to the user's question about Hamlet. I need to clarify this mismatch to prevent any confusion."},
   {'reasoning': "Ensuring transparency about the document discrepancy is crucial for maintaining user trust. I should clearly state that the retrieved documents are not relevant to the user's query."},
   {'reasoning': 'To enhance user satisfaction and understanding, I should consider asking a follow-up question to confirm if the user nee

In [5]:
shakespeare_agent.prompt_context.history

[{'role': 'user',
  'content': 'Tasks: Your are assigned with following tasks:\nPerceive the current state of the chat environment:\n\n## Environment State\n<environment_state>\n-   content: What is Hamlet\'s famous \'To be or not to be\' soliloquy about?\n  timestamp: 2025-01-23 05:55:45\n  role: user\n</environment_state>\n\n## Recent Cognitive Memories\n<cognitive_memories>\n-   cognitive_step: action\n  content: {"agent_id": "agent_007", "action": {"thoughts": [{"reasoning": "The user has asked about Hamlet\'s \'To be or not to be\' soliloquy, which is a significant part of the play \'Hamlet\' by William Shakespeare. This soliloquy explores existential themes, pondering the nature of existence and the question of life and death."}, {"reasoning": "The documents retrieved are from \'Antony and Cleopatra\', which is a different play by Shakespeare and does not directly relate to the user\'s question about Hamlet."}, {"reasoning": "To address the user\'s query effectively, I will provi

In [11]:
# Generate a response
perception_prompt = await shakespeare_agent.perceive(environment_name="chat", return_prompt=True)
print(json.dumps(perception_prompt.model_dump_json(), indent=4))


Cognitive Memory Results:
Memory 1:
{'cognitive_step': 'action', 'content': '{"agent_id": "shakespeare_agent", "action": {"thoughts": [{"reasoning": "The user is asking about Hamlet\'s \'To be or not to be\' soliloquy, which is a well-known passage that explores deep existential themes. It is important to provide a clear and concise summary that emphasizes these themes of existence, life, and death."}, {"reasoning": "There is a discrepancy between the user\'s question and the retrieved documents, which are from \'Antony and Cleopatra,\' a different play by Shakespeare. I need to address this mismatch to prevent confusion."}, {"reasoning": "Ensuring transparency about the document discrepancy is crucial for maintaining user trust. I should clearly state that the retrieved documents are not relevant to the user\'s query."}, {"reasoning": "To enhance user satisfaction and understanding, I should consider asking a follow-up question to confirm if the user needs further clarification or ha

/Users/interstellarninja/miniconda3/envs/market-agents/lib/python3.12/site-packages/pydantic/main.py:415: UserWarning: Pydantic serializer warnings:
  Expected `Union[str, generator]` but got `list` - serialized value may not be as expected
  Expected `Union[str, generator]` but got `list` - serialized value may not be as expected
  Expected `Union[str, generator]` but got `list` - serialized value may not be as expected
  Expected `Union[str, generator]` but got `list` - serialized value may not be as expected
  Expected `Union[str, generator]` but got `list` - serialized value may not be as expected
  Expected `Union[str, generator]` but got `list` - serialized value may not be as expected
  Expected `Union[str, generator]` but got `list` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(


In [12]:
print(perception_prompt.new_message)

Tasks: Your are assigned with following tasks:
Perceive the current state of the chat environment:

## Environment State
<environment_state>
-   content: What is Hamlet's famous 'To be or not to be' soliloquy about?
  timestamp: 2025-01-23 05:55:45
  role: user
</environment_state>

## Recent Cognitive Memories
<cognitive_memories>
-   cognitive_step: action
  content: {"agent_id": "shakespeare_agent", "action": {"thoughts": [{"reasoning": "The user is asking about Hamlet's 'To be or not to be' soliloquy, which is a well-known passage that explores deep existential themes. It is important to provide a clear and concise summary that emphasizes these themes of existence, life, and death."}, {"reasoning": "There is a discrepancy between the user's question and the retrieved documents, which are from 'Antony and Cleopatra,' a different play by Shakespeare. I need to address this mismatch to prevent confusion."}, {"reasoning": "Ensuring transparency about the document discrepancy is crucial

In [17]:
print(json.dumps(perception_prompt.history, indent=4))

[
    {
        "role": "user",
        "content": "Tasks: Your are assigned with following tasks:\nPerceive the current state of the chat environment:\n\n## Environment State\n<environment_state>\n-   content: What is Hamlet's famous 'To be or not to be' soliloquy about?\n  timestamp: 2025-01-23 05:55:45\n  role: user\n</environment_state>\n\n## Recent Cognitive Memories\n<cognitive_memories>\n-   cognitive_step: action\n  content: {\"agent_id\": \"agent_007\", \"action\": {\"thoughts\": [{\"reasoning\": \"The user has asked about Hamlet's 'To be or not to be' soliloquy, which is a significant part of the play 'Hamlet' by William Shakespeare. This soliloquy explores existential themes, pondering the nature of existence and the question of life and death.\"}, {\"reasoning\": \"The documents retrieved are from 'Antony and Cleopatra', which is a different play by Shakespeare and does not directly relate to the user's question about Hamlet.\"}, {\"reasoning\": \"To address the user's quer

In [18]:
print(perception_prompt.system_message)

{'role': 'system', 'content': 'Role: You are a agent agent. You are a long-horizon task execution agent based on verbal reinforcement learning. The current date and time is 2025-01-23 06:46:44.\n\n'}


In [19]:
print(perception_prompt.structured_output)

json_schema={'properties': {'monologue': {'description': "Agent's internal monologue about the perceived environment", 'title': 'Monologue', 'type': 'string'}, 'key_observations': {'description': "Agent's key observations from the environment", 'items': {'type': 'string'}, 'title': 'Key Observations', 'type': 'array'}, 'strategy': {'description': "Agent's strategies given the current environment", 'items': {'type': 'string'}, 'title': 'Strategy', 'type': 'array'}, 'confidence': {'description': 'Confidence score between 0.0 and 1.0', 'maximum': 1.0, 'minimum': 0.0, 'title': 'Confidence', 'type': 'number'}}, 'required': ['monologue', 'key_observations', 'strategy', 'confidence'], 'title': 'PerceptionSchema', 'type': 'object'} schema_name='generate_structured_output' schema_description='Generate a structured output based on the provided JSON schema.' instruction_string='Please follow this JSON schema for your response:' strict_schema=False schema_instruction='Please follow this JSON schem